In [ ]:
!pwd

import sys
import os

# Detect if we are in Colab or a generic cloud environment (like Vast.ai)
is_colab = 'google.colab' in sys.modules 
is_vast = os.path.exists('/root/.vast_container') # Common Vast.ai indicator
is_cloud = is_colab or is_vast

if is_cloud:
  py_url = "https://raw.githubusercontent.com/fanaustinca/so101ai/refs/heads/main/lerobot_setup.py"

  !curl -O {py_url}
  if os.path.exists("lerobot_setup.py"):
    print("\nSuccessfully downloaded lerobot_setup.py")
  else:
    print("\nDownload failed.")
else:
  print("Running on local machine. Skipping cloud setup.")

In [ ]:
# Setting lerobot env
from lerobot_setup import setup_lerobot_env

setup_lerobot_env()

In [ ]:
# Prepare training data set. Combine multiple batches under one training data.
from datasets import load_dataset, concatenate_datasets

# Generate input_datasets using a loop
input_datasets = [f'fanaustinca/fll_diffusion_v2_batch_{i:02d}' for i in range(1, 3)]

output_dataset_name = 'fanaustinca/fll_training_400episodes_test'

# Load datasets
print("Loading datasets...")
datasets_list = []
for dataset_name in input_datasets:
    print(f"  Loading {dataset_name}")
    dataset = load_dataset(dataset_name, trust_remote_code=True)
    # Assuming the dataset is a DatasetDict and we want to combine the 'train' split
    # Adjust if your dataset structure is different (e.g., just a Dataset object)
    if 'train' in dataset:
        datasets_list.append(dataset['train'])
    else:
        datasets_list.append(dataset)

# Concatenate datasets
print("Concatenating datasets...")
combined_dataset = concatenate_datasets(datasets_list)

# Push to Hugging Face Hub
print(f"Pushing combined dataset to {output_dataset_name}...")
combined_dataset.push_to_hub(output_dataset_name, private=False)
print("Dataset combined and pushed successfully!")

In [ ]:
from lerobot_setup import cd_lerobot
import json
import sys
from IPython.display import display, Markdown

cd_lerobot()

# Training ACT model.
# Adjust the following variables before starting training
tag = '400episodes'
policy_type = 'diffusion'
n_action_steps = 10 if policy_type == 'diffusion' else 100
hf_namespace = 'fanaustinca'
training_repo = f"{hf_namespace}/fll_training_combined"
model_repo = f"{hf_namespace}/fll_diffusion_model_400episodes_v2"
# Use 'local_output' consistently
local_output = f"outputs/model/{model_repo}"
steps = 150000
batch_size = 64
num_workers = 16
enable_image_transforms = True

if policy_type not in ['act', 'diffusion']:
    raise ValueError(f"policy_type can only be 'act' or 'diffusion', got: {policy_type}")


# Clean up output from last run
!rm -rf {local_output}

tfs_config = {
  "to_tensor": {"type": "ToImage"},
  "affine": {
    "type": "RandomAffine",
    "kwargs": {"degrees": 0, "translate": [0.05, 0.05]}
  }
}

"""
    --dataset.image_transforms.enable=true \
    --dataset.image_transforms.random_order=false \
    --dataset.image_transforms.tfs='{tfs_string}' \
"""

# Convert to a compact JSON string for the CLI
tfs_string = json.dumps(tfs_config, separators=(',', ':'))

arguments = f"""\
    --policy.type={policy_type} \
    --dataset.repo_id={training_repo} \
    --job_name=fll_training_{policy_type} \
    --output_dir={local_output} \
    --policy.n_action_steps={n_action_steps} \
    --policy.device=cuda \
    --steps={steps} \
    --save_freq=5000 \
    --batch_size={batch_size} \
    --log_freq=100 \
    --num_workers={num_workers} \
    --policy.use_amp=true \
    --policy.push_to_hub=true \
    --policy.repo_id={model_repo} \
    --dataset.image_transforms.enable=true \
    --dataset.image_transforms.random_order=true \
    --wandb.enable=true"""

if enable_image_transforms:
    arguments += " --dataset.image_transforms.enable=true \
      --dataset.image_transforms.random_order=true"

if policy_type == 'diffusion':
    arguments += " --policy.num_inference_steps=30"

# Your variables
command_str = f"{sys.executable} src/lerobot/scripts/lerobot_train.py {arguments}"

# Wrap it in markdown code blocks
display(Markdown(f"```bash\n{command_str}\n```"))

command_str = f"hf repo tag create {model_repo} {tag} -m \"Diffusion model trained on {training_repo}, tag {tag}\""
display(Markdown(f"```bash\n{command_str}\n```"))
